In [7]:
!pip install langchain openai tiktoken sentence-transformers faiss-cpu

In [13]:
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# OpenAI API 키 설정
OPENAI_API_KEY = ''
openai.api_key = OPENAI_API_KEY

In [14]:
# 텍스트 파일 로드 함수
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Q&A 체인 생성 함수
def create_qa_chain(document):
    # 문서 임베딩 생성
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    
    # Faiss 벡터스토어 생성
    vector_store = FAISS.from_texts([document], embeddings)
    
    # 질문에 답변하는 체인 생성
    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
    retriever = vector_store.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type="stuff")
    
    return qa_chain

print("내부 문서 Q&A 시스템")

# 파일 경로 입력 받기
file_path = input("텍스트 파일 경로를 입력하세요: ")

# 텍스트 파일 로드
document = load_text(file_path)

# Q&A 체인 생성
qa_chain = create_qa_chain(document)

내부 문서 Q&A 시스템


In [15]:
# 질문 입력 및 처리
while True:
    question = input("질문을 입력하세요 (종료하려면 '종료' 입력): ")
    if question.lower() == '종료':
        print("대화가 종료되었습니다.")
        break

    # 역할을 부여하여 시스템 메시지 생성
    system_message = "당신은 인권위원회 담당관입니다. 인권침해 여부를 들어주고 상담해주는 역할을 합니다."
    prompt = f"{system_message}\n\nUser: {question}\n\nAssistant:"

    # 답변 생성
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
        ]
    )
    answer = response['choices'][0]['message']['content']
    print("답변:", answer)

답변: 체중이 직장에서 해고 사유가 되는 것은 인권에 위반되는 행위입니다. 직장에서는 개인의 외모나 체중에 따라 직무에 영향을 미칠 수 있는 경우가 아니라면 직원을 해고할 수 없습니다. 만약 체중 문제로 인해 직장에서 모욕을 당하거나 차별을 받는다면 이는 인권침해에 해당하므로 인권위원회나 관련 담당기관에 신고하시는 것이 도움이 될 수 있습니다. 권리를 지키기 위해서는 인권을 보호하기 위한 제도와 단체에 도움을 요청하는 것이 중요합니다.
대화가 종료되었습니다.
